# 도구 (Tools)

도구(Tool)는 에이전트, 체인 또는 LLM이 외부 세계와 상호작용하기 위한 인터페이스입니다.

LangChain 에서 기본 제공하는 도구를 사용하여 쉽게 도구를 활용할 수 있으며, 사용자 정의 도구(Custom Tool) 를 쉽게 구축하는 것도 가능합니다.

**LangChain 에 통합된 도구 리스트는 아래 링크에서 확인할 수 있습니다.**

- [LangChain 통합된 도구 리스트](https://python.langchain.com/v0.1/docs/integrations/tools/)

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Tools")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Tools


In [3]:
import warnings

# 경고 메시지 무시
warnings.filterwarnings("ignore")

## 빌트인 도구(built-in tools)

랭체인에서 제공하는 사전에 정의된 도구(tool) 와 툴킷(toolkit) 을 사용할 수 있습니다.

tool 은 단일 도구를 의미하며, toolkit 은 여러 도구를 묶어서 하나의 도구로 사용할 수 있습니다.

관련 도구는 아래의 링크에서 참고하실 수 있습니다.

**참고**
- [LangChain Tools/Toolkits](https://python.langchain.com/docs/integrations/tools/)

### Python REPL 도구

이 도구는 Python 코드를 REPL(Read-Eval-Print Loop) 환경에서 실행하기 위한 두 가지 주요 클래스를 제공합니다
- PythonREPLTool

**설명**

- Python 셸 환경을 제공합니다.
- 유효한 Python 명령어를 입력으로 받아 실행합니다.
- 결과를 보려면 print(...) 함수를 사용해야 합니다.

**주요 특징**

- sanitize_input: 입력을 정제하는 옵션 (기본값: True)
- python_repl: PythonREPL 인스턴스 (기본값: 전역 범위에서 실행)

**사용 방법**

- PythonREPLTool 인스턴스 생성
- run 또는 arun, invoke 메서드를 사용하여 Python 코드 실행

**입력 정제**

- 입력 문자열에서 불필요한 공백, 백틱, 'python' 키워드 등을 제거합니다.

In [4]:
from langchain_experimental.tools import PythonREPLTool

# 파이썬 코드를 실행하는 도구를 생성합니다.
python_tool = PythonREPLTool()

In [5]:
# 파이썬 코드를 실행하고 결과를 반환합니다.
print(python_tool.invoke("print(100 + 200)"))

Python REPL can execute arbitrary code. Use with caution.


300



아래는 LLM 에게 파이썬 코드를 작성하도록 요청하고 결과를 반환하는 예제입니다.

**흐름 정리**
1. LLM 모델에게 특정 작업을 수행하는 Python 코드를 작성하도록 요청합니다.
2. 작성된 코드를 실행하여 결과를 얻습니다.
3. 결과를 출력합니다.


In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


# 파이썬 코드를 실행하고 중간 과정을 출력하고 도구 실행 결과를 반환하는 함수
def print_and_execute(code, debug=True):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)


# 파이썬 코드를 작성하도록 요청하는 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human", "{input}"),
    ]
)
# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 프롬프트와 LLM 모델을 사용하여 체인 생성
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

In [7]:
# 결과 출력
print(chain.invoke("로또 번호 생성기를 출력하는 코드를 작성하세요."))

CODE:
import random

def generate_lotto_numbers():
    return sorted(random.sample(range(1, 46), 6))

print(generate_lotto_numbers())
[3, 6, 9, 14, 15, 35]



### 검색 API 도구

Tavily 검색 API를 활용하여 검색 기능을 구현하는 도구입니다. 이 도구는 두 가지 주요 클래스를 제공합니다: `TavilySearchResults`와 `TavilyAnswer`.

**API 키 발급 주소**
- https://app.tavily.com/

발급한 API 키를 환경변수에 설정합니다.

`.env` 파일에 아래와 같이 설정합니다.

```
TAVILY_API_KEY=tvly-abcdefghijklmnopqrstuvwxyz
```

### TavilySearchResults

**설명**
- Tavily 검색 API를 쿼리하고 JSON 형식의 결과를 반환합니다.
- 포괄적이고 정확하며 신뢰할 수 있는 결과에 최적화된 검색 엔진입니다.
- 현재 이벤트에 대한 질문에 답변할 때 유용합니다.

**주요 매개변수**
- `max_results` (int): 반환할 최대 검색 결과 수 (기본값: 5)
- `search_depth` (str): 검색 깊이 ("basic" 또는 "advanced")
- `include_domains` (List[str]): 검색 결과에 포함할 도메인 목록
- `exclude_domains` (List[str]): 검색 결과에서 제외할 도메인 목록
- `include_answer` (bool): 원본 쿼리에 대한 짧은 답변 포함 여부
- `include_raw_content` (bool): 각 사이트의 정제된 HTML 콘텐츠 포함 여부
- `include_images` (bool): 쿼리 관련 이미지 목록 포함 여부

**반환 값**
- 검색 결과를 포함하는 JSON 형식의 문자열(url, content)

혹은 아래의 주석을 해제하고 발급받은 API 키를 입력합니다.

In [8]:
# import os

# os.environ["TAVILY_API_KEY"] = "TAVILY API 키 입력"

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 생성
tool = TavilySearchResults(
    max_results=6,
    include_answer=True,
    include_raw_content=True,
    # include_images=True,
    # search_depth="advanced", # or "basic"
    include_domains=["github.io", "wikidocs.net"],
    # exclude_domains = []
)

In [10]:
# 도구 실행
tool.invoke({"query": "LangChain Tools 에 대해서 알려주세요"})

[{'url': 'https://teddylee777.github.io/langchain/langchain-agent/',
  'content': '태그:\nAgent,\nAPI KEY,\nFAISS,\nLangChain,\nLangSmith,\nmemory,\nOpenAI,\nPython,\nRetriever,\nTavily Search,\ntools,\n검색도구,\n랭체인,\n에이전트\n카테고리:\nlangchain\n업데이트: 2024년 02월 09일\n참고\nLangChain RAG 파헤치기: 다음의 추적 링크에서 자세한 단계별 수행 결과를 확인할 수 있습니다\nLangSmith 추적\n다음의 추적 링크에서 자세한 단계별 수행 결과를 확인할 수 있습니다\nLangSmith 추적\n④ 메모리 추가하기\n앞서 언급했듯이, 이 에이전트는 상태가 없습니다. LangChain 한국어 튜토리얼\n바로가기 👀\n[LangChain] 에이전트(Agent)와 도구(tools)를 활용한 지능형 검색 시스템 구축 가이드\n2024년 02월 09일\n41 분 소요\n이 글에서는 LangChain 의 Agent 프레임워크를 활용하여 복잡한 검색과 📍 전체 템플릿 코드\n다음의 추적 링크에서 자세한 단계별 수행 결과를 확인할 수 있습니다\nLangSmith 추적\n마무리입니다!\n 문서 기반 QA 시스템 설계 방법 - 심화편\n2024년 02월 06일\n22 분 소요\nLangChain의 RAG 시스템을 통해 문서(PDF, txt, 웹페이지 등)에 대한 질문-답변을 찾는 과정을 정리하였습니다.\n'},
 {'url': 'https://13akstjq.github.io/TIL/post/2024-07-09-LLMStudyDiaryComprehensiveReviewofLangChainPart4',
  'content': 'Memory. 원래 코드는 그대로 작동합니다. Langchain의 다양한 메모리 유형 목록에 대한 링크가 깨져 있어 찾아보았습니다. ... tools=toolkit

### Image 생성 도구 (DALL-E)

- `DallEAPIWrapper 클래스`: OpenAI의 DALL-E 이미지 생성기를 위한 래퍼(wrapper)입니다.

이 도구를 사용하면 DALL-E API를 쉽게 통합하여 텍스트 기반 이미지 생성 기능을 구현할 수 있습니다. 다양한 설정 옵션을 통해 유연하고 강력한 이미지 생성 도구로 활용할 수 있습니다.

**주요 속성**

- `model`: 사용할 DALL-E 모델 이름 (기본값: "dall-e-2", "dall-e-3")

- `n`: 생성할 이미지 수 (기본값: 1)

- `size`: 생성할 이미지 크기
  - "dall-e-2": "1024x1024", "512x512", "256x256"
  - "dall-e-3": "1024x1024", "1792x1024", "1024x1792"

- `style`: 생성될 이미지의 스타일 (기본값: "natural", "vivid")

- `quality`: 생성될 이미지의 품질 (기본값: "standard", "hd")

- `max_retries`: 생성 시 최대 재시도 횟수

**주요 기능**
- DALL-E API를 사용하여 텍스트 설명에 기반한 이미지 생성

**흐름 정리**

다음은 DALL-E Image Generator 를 사용하여 이미지를 생성하는 예제입니다.

이번에는 `DallEAPIWrapper` 를 사용하여 이미지를 생성해 보겠습니다.

이때 입력 프롬프트는 LLM 모델에게 이미지를 생성하는 프롬프트를 작성하도록 요청합니다.

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9, max_tokens=1000)

# DALL-E 이미지 생성을 위한 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "
    "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"
    "Output should be less than 1000 characters. Write in English only."
    "Image Description: \n{image_desc}",
)

# 프롬프트, LLM, 출력 파서를 연결하는 체인 생성
chain = prompt | llm | StrOutputParser()

# 체인 실행
image_prompt = chain.invoke(
    {"image_desc": "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"}
)

# 이미지 프롬프트 출력
print(image_prompt)

A neo-classical painting depicting a group of individuals in a grand, opulent room, all intently gazing at their smartphones instead of engaging with one another. The scene is rich in classical architecture, featuring marble columns and intricate moulding, with a large window allowing soft, natural light to illuminate the space. The people, dressed in elegant, period-appropriate attire, display expressions of fascination and distraction, contrasting sharply with the traditional setting. Some individuals are seated on lavishly upholstered chairs, while others stand, their poses reminiscent of classical sculptures. Elements of modern technology, such as the glowing screens of the smartphones, are juxtaposed against the timeless beauty of the room, highlighting the irony of contemporary life clashing with neo-classical themes. The color palette should be warm and inviting, with deep golds, rich reds, and soft whites, alluding to a blend of past elegance and present absurdity.


그럼, 이전에 생성한 이미지 프롬프트를 `DallEAPIWrapper` 에 입력하여 이미지를 생성해 보겠습니다.

In [12]:
# DALL-E API 래퍼 가져오기
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from IPython.display import Image

# DALL-E API 래퍼 초기화
# model: 사용할 DALL-E 모델 버전
# size: 생성할 이미지 크기
# quality: 이미지 품질
# n: 생성할 이미지 수
dalle = DallEAPIWrapper(model="dall-e-3", size="1024x1024", quality="standard", n=1)

# 질문
query = "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"

# 이미지 생성 및 URL 받기
# chain.invoke()를 사용하여 이미지 설명을 DALL-E 프롬프트로 변환
# dalle.run()을 사용하여 실제 이미지 생성
image_url = dalle.run(chain.invoke({"image_desc": query}))

# 생성된 이미지를 표시합니다.
Image(url=image_url, width=500)

## 사용자 정의 도구(Custom Tool)

LangChain 에서 제공하는 빌트인 도구 외에도 사용자가 직접 도구를 정의하여 사용할 수 있습니다.

이를 위해서는 `langchain.tools` 모듈에서 제공하는 `tool` 데코레이터를 사용하여 함수를 도구로 변환합니다.

### @tool 데코레이터

이 데코레이터는 함수를 도구로 변환하는 기능을 제공합니다. 다양한 옵션을 통해 도구의 동작을 커스터마이즈할 수 있습니다.

**사용 방법**
1. 함수 위에 `@tool` 데코레이터 적용
2. 필요에 따라 데코레이터 매개변수 설정

이 데코레이터를 사용하면 일반 Python 함수를 강력한 도구로 쉽게 변환할 수 있으며, 자동화된 문서화와 유연한 인터페이스 생성이 가능합니다.

In [13]:
from langchain.tools import tool


# 데코레이터를 사용하여 함수를 도구로 변환합니다.
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [14]:
# 도구 실행
add_numbers.invoke({"a": 3, "b": 4})

7

In [15]:
# 도구 실행
multiply_numbers.invoke({"a": 3, "b": 4})

12

### 구글 뉴스기사 검색 도구

`langchain-teddynote` 패키지에서 제공하는 `GoogleNews` 도구를 사용하여 구글 뉴스기사를 검색하는 도구입니다.

**참고**
- API 키가 필요하지 않습니다. (RSS 피드를 사용하기 때문)

news.google.com 에서 제공하는 뉴스기사를 검색하는 도구입니다.

**설명**
- 구글 뉴스 검색 API를 사용하여 최신 뉴스를 검색합니다.
- 키워드를 기반으로 뉴스를 검색할 수 있습니다.
- 최신 뉴스를 검색할 수 있습니다.

**주요 매개변수**
- `k` (int): 반환할 최대 검색 결과 수 (기본값: 5)


사용하기 전 패키지를 업데이트 해주세요.

In [16]:
# !pip install -qU langchain-teddynote

In [17]:
from langchain_teddynote.tools import GoogleNews

# 도구 생성
news_tool = GoogleNews()

In [18]:
# 최신 뉴스 검색
news_tool.search_latest(k=5)

[{'url': 'https://news.google.com/rss/articles/CBMiZkFVX3lxTE55OWlTNnNaWi03N2FoQThLRHM4aktxNmZNenA5czJPMmhmWFZ0WFUxUXZXQ09EUnZCREdULVlGOGYxVElUanJoRFR6aHdCNWQxNk1GQ2x2Y2stX1YxTXN0S2d2WFI2Zw?oc=5',
  'content': '조규홍 복지장관 “의대 5년 단축, 교육부와 사전 협의 없어” - 한겨레'},
 {'url': 'https://news.google.com/rss/articles/CBMiW0FVX3lxTE82V1pJQ29nOE1aZWFhMUc3Wk8xbWFhVGFnQ25zOTU2b2I1TzktY0dZT0JRSkZ4RE91YWJ6NVhKNHlEMWJGSURtbDV6aUREZlUxQU1zbkwwMG9ZdWM?oc=5',
  'content': '한 발 물러선 정부…의대생 복귀 가능성은 ‘불투명’ - KBS뉴스'},
 {'url': 'https://news.google.com/rss/articles/CBMiZEFVX3lxTE9zOE52SDloVEM2TTBZMEQ2N3NYV0VxX3VuRWl2cTEydWk1ZGtqUS1IN1FzckFCMTNqZ045RF9yWU1Yd3k5ejdrSEIweTl3bHFtaTh6Y2pHckZnWEQ2ZjQ3MDVWVjI?oc=5',
  'content': '검경·국세청·감사원, 방통위에 18명 파견…“특별수사본부로 전락” - 한겨레'},
 {'url': 'https://news.google.com/rss/articles/CBMid0FVX3lxTE0tVmNIM3A0eldlRHdFZURKRV9FZk4zdjBVWFMtcXRtRDVhRExxYTRveG1EclN5RnNNTDJ4U1U5czFQdzJCc094clpVTjNzR0dGbHpFYXFQenJwWDNGMUp4Y0pOaU1TQ2l5eDBGRFRBR2JzeUZyRUtv0gF3QVVfeXFMTzAzUkhBRzE2b01KVXY1VGkxaUoxYUdL

In [19]:
# 키워드로 뉴스 검색
news_tool.search_by_keyword("AI 투자", k=5)

[{'url': 'https://news.google.com/rss/articles/CBMiTkFVX3lxTE5xUEdrNHZZclQ4RUNmNGpqZG1COUNfckVOZ2hqQTJ4NGkwa0NRU0Q1azRQNzMwTTBJVDloQmNKamRZc3dic0plQWtEUTNXUQ?oc=5',
  'content': 'AI 투자 늘리는 통신사, 주파수 재할당 대가 마련 부담 - 전자신문'},
 {'url': 'https://news.google.com/rss/articles/CBMiU0FVX3lxTFB1VDRjTlIxeHRjV3NKaERHMFJLOFNpUlNUVnZzQ0JvOUpISVE2cW9rbTZPVU9lQXZuUURJNHF0aDZyaDdQM0F5NFhsc0NrN1dHNEZz?oc=5',
  'content': '글로벌 CEO들 "향후 3년 무조건 AI 투자···직원도 더 뽑을 것" - 네이트 뉴스'},
 {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE9BQUtjd3pDS1dXRkZQTlo4Y2w2d2laZFFQRXdsbGlCMFRTcWJYLTNLaksxbFlnYzk3clVnYlFxZF93U2xubXY1VVhBTDQ2SDhLbm9hcDdXQ3prTlJtV0VCYTFINmNRbjR5N3c?oc=5',
  'content': 'MIT 경제학자 "10년간 AI에 영향받을 직업은 5% 불과...기업은 투자비만 날리게 될 것" - AI타임스'},
 {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE05SHBCX2FSa2Fmekl3WDJkdW03NWxXT2lVM0o0TVlKeTJPLVA3R1BvRWJ6R282TFRZX1RqZWNBejVHWnc1a05fS2d3Y2FOWDQ1ZUpDT2l6TTgyMlpiR2ZiWXVvTkp2OTNGU2c?oc=5',
  'content': 'AI로 위조품 가려내는 마크비전, 220억 투자 유치 - 한국일보'},
 {'ur

In [20]:
from langchain_teddynote.tools import GoogleNews
from langchain.tools import tool
from typing import List, Dict


# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

In [21]:
# 실행 결과
search_keyword.invoke({"query": "AI 투자"})

AI 투자


[{'url': 'https://news.google.com/rss/articles/CBMiTkFVX3lxTE5xUEdrNHZZclQ4RUNmNGpqZG1COUNfckVOZ2hqQTJ4NGkwa0NRU0Q1azRQNzMwTTBJVDloQmNKamRZc3dic0plQWtEUTNXUQ?oc=5',
  'content': 'AI 투자 늘리는 통신사, 주파수 재할당 대가 마련 부담 - 전자신문'},
 {'url': 'https://news.google.com/rss/articles/CBMiU0FVX3lxTFB1VDRjTlIxeHRjV3NKaERHMFJLOFNpUlNUVnZzQ0JvOUpISVE2cW9rbTZPVU9lQXZuUURJNHF0aDZyaDdQM0F5NFhsc0NrN1dHNEZz?oc=5',
  'content': '글로벌 CEO들 "향후 3년 무조건 AI 투자···직원도 더 뽑을 것" - 네이트 뉴스'},
 {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE9BQUtjd3pDS1dXRkZQTlo4Y2w2d2laZFFQRXdsbGlCMFRTcWJYLTNLaksxbFlnYzk3clVnYlFxZF93U2xubXY1VVhBTDQ2SDhLbm9hcDdXQ3prTlJtV0VCYTFINmNRbjR5N3c?oc=5',
  'content': 'MIT 경제학자 "10년간 AI에 영향받을 직업은 5% 불과...기업은 투자비만 날리게 될 것" - AI타임스'},
 {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE05SHBCX2FSa2Fmekl3WDJkdW03NWxXT2lVM0o0TVlKeTJPLVA3R1BvRWJ6R282TFRZX1RqZWNBejVHWnc1a05fS2d3Y2FOWDQ1ZUpDT2l6TTgyMlpiR2ZiWXVvTkp2OTNGU2c?oc=5',
  'content': 'AI로 위조품 가려내는 마크비전, 220억 투자 유치 - 한국일보'},
 {'ur